In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import re
from utils_rec import precision_at_k,random_recommendation,popularity_recommendation
# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
# from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [157]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
data = pd.read_csv('../data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [5]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    items = items_weights.item_id.tolist()
    p=items_weights.weight.tolist()
    recs = np.random.choice(items, size=n, replace=False,p=p)
    
    return recs.tolist()

In [7]:
%%time

# # your_code

items_weights=data_train.groupby('item_id', as_index=False).sales_value.sum()+0.01
items_weights['weight']=items_weights.sales_value.apply(np.log1p)/items_weights.sales_value.apply(np.log1p).sum()
items_weights.drop('sales_value', axis=1, inplace=True)
weighted_random_recommendation(items_weights)

Wall time: 263 ms


[15682370.01, 15596195.01, 1200891.01, 12810351.01, 1112665.01]

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [138]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [141]:
# your_code


In [117]:
for column in result.columns[1:]:
    result[column]=result[column].apply(lambda x: list(map(int,(re.findall('\d+',x)))))
    


In [127]:
for column in result.columns[2:]:
    res=result.apply(lambda row: precision_at_k(row[column], row['actual']), axis=1).mean()
    print(column, res)

random_recommendation 0.0005876591576885406
popular_recommendation 0.15523996082272082
itemitem 0.033594515181195064
cosine 0.03525954946131262
tfidf 0.03614103819784544
own_purchases 0.1799869409076044


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code

In [143]:
data = pd.read_csv('../data/retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [149]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [150]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [151]:
items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1107152, 1024875, 922071, 1139336, 6534246]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13039636, 6773215, 9553382, 524165, 64754]"


In [152]:
items = popularity_recommendation(data_train, n=5000)

result['random_recommendation_5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

,user_id,actual,random_recommendation,random_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1107152, 1024875, 922071, 1139336, 6534246]","[1076587, 1028938, 9527323, 5585510, 5591170]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13039636, 6773215, 9553382, 524165, 64754]","[912141, 990869, 10284966, 1096261, 1005456]"


In [153]:
for column in result.columns[2:]:
    res=result.apply(lambda row: precision_at_k(row[column], row['actual']), axis=1).mean()
    print(column, res)

random_recommendation 0.0005876591576885406
random_recommendation_5000 0.00656219392752203


In [155]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [164]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [158]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()


In [159]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [175]:
%%time

for k in range(1,11):
    model = ItemItemRecommender(K=k, num_threads=4) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=False)

    recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                            user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                            N=5, # кол-во рекомендаций 
                            filter_already_liked_items=False, 
                            filter_items=None, 
                            recalculate_user=True)
    
    result['itemitem'+str(k)] = result['user_id'].apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

Wall time: 12.1 s


In [176]:
for column in result.columns[2:]:
    res=result.apply(lambda row: precision_at_k(row[column], row['actual']), axis=1).mean()
    print(column,'  ',res)

itemitem1    0.1799869409076044
itemitem2    0.1920094678419821
itemitem3    0.1860920666013684
itemitem4    0.14495592556317158
itemitem5    0.13692458374142857
itemitem6    0.14201762977472907
itemitem7    0.1448579823702236
itemitem8    0.14720861900097773
itemitem9    0.14848188050930292
itemitem10    0.15093046033300497
itemitem11    0.15220372184133013
itemitem12    0.1533790401567072
itemitem13    0.15318315377081107
itemitem14    0.1534769833496553
itemitem15    0.15318315377081096
itemitem16    0.1533790401567071
itemitem17    0.15318315377081096
itemitem18    0.15337904015670714
itemitem19    0.15337904015670714


In [172]:
# k=2,3 - оптимальные варианты